In [1]:
import os
import numpy as np
import torch
import nibabel as nib
import pandas as pd
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from tqdm import tqdm
import matplotlib.pyplot as plt
import re 

In [2]:


def load_nifti(path):
    return nib.load(path).get_fdata().astype(np.float32)

def normalize(img):
    img = img - np.min(img)
    if np.max(img) > 0:
        img = img / np.max(img)
    return img

def compute_3d_metrics(real, fake):
    real = normalize(real)
    fake = normalize(fake)

    ssim_val = ssim(real, fake, data_range=1.0)
    psnr_val = psnr(real, fake, data_range=1.0)
    mae = np.mean(np.abs(real - fake))
    mse = np.mean((real - fake) ** 2)

    return {
        'SSIM': round(ssim_val, 4),
        'PSNR': round(psnr_val, 2),
        'MAE': round(mae, 5),
        'MSE': round(mse, 5)
    }

def save_metrics_to_excel(results, output_path='similarity_metrics.xlsx'):
    df = pd.DataFrame.from_dict(results, orient='index')
    df.index.name = 'Sample ID'
    df.reset_index(inplace=True)
    df.to_excel(output_path, index=False)
    print(f"\n✅ Saved metrics to Excel: {output_path}")
    return df  # return the dataframe for plotting



In [3]:
def crop_or_pad_to_match_3d(real, target_shape):
    """
    Adjust the 3D `real` volume to match `target_shape` using symmetric padding and center cropping.
    All volumes are assumed to be 3D: (D, H, W)
    """
    cd, ch, cw = target_shape
    d, h, w = real.shape

    # Padding if real image is smaller
    pad_d = max(cd - d, 0)
    pad_h = max(ch - h, 0)
    pad_w = max(cw - w, 0)

    pad_width = (
        (pad_d // 2, pad_d - pad_d // 2),
        (pad_h // 2, pad_h - pad_h // 2),
        (pad_w // 2, pad_w - pad_w // 2),
    )

    if pad_d > 0 or pad_h > 0 or pad_w > 0:
        real = np.pad(real, pad_width, mode='constant', constant_values=0)

    # Crop center if real image is larger
    d, h, w = real.shape
    start_d = (d - cd) // 2
    start_h = (h - ch) // 2
    start_w = (w - cw) // 2

    end_d = start_d + cd
    end_h = start_h + ch
    end_w = start_w + cw

    return real[start_d:end_d, start_h:end_h, start_w:end_w]


In [4]:
def get_files_by_creation_order(directory, extensions=('.nii', '.nii.gz')):
    files = [
        f for f in os.listdir(directory)
        if f.endswith(extensions)
    ]
    # Sort by creation time (earliest first)
    files.sort(key=lambda x: os.path.getctime(os.path.join(directory, x)))
    return files

In [5]:
import os
import nibabel as nib
import numpy as np
from tqdm import tqdm

def load_nifti(path):
    return nib.load(path).get_fdata()
import os
from tqdm import tqdm

def evaluate_all(fake_dir, real_dir):
    results = {}
    fake_files = sorted([f for f in os.listdir(fake_dir) if f.endswith('.nii')])

    for fake_file in tqdm(fake_files, desc="Evaluating"):
        # Extract index as the part before the first underscore
        index = os.path.splitext(fake_file)[0].split('_')[0]
        real_file = f"test-volume-{index}.nii"

        fake_path = os.path.join(fake_dir, fake_file)
        real_path = os.path.join(real_dir, real_file)

        print(f"Comparing fake: {fake_file}  <-->  real: {real_file}")

        if not os.path.exists(real_path):
            print(f"⚠️ Real file not found for {fake_file}")
            continue

        try:
            fake_img = load_nifti(fake_path)
            real_img = load_nifti(real_path)

            if fake_img.shape != real_img.shape:
                try:
                    real_img = crop_or_pad_to_match_3d(real_img, fake_img.shape)
                except Exception as e:
                    print(f"❌ Could not crop/pad real image for {fake_file}: {e}")
                    continue

            metrics = compute_3d_metrics(real_img, fake_img)
            results[fake_file] = metrics

        except Exception as e:
            print(f"❌ Error processing {fake_file}: {e}")

    return results


In [6]:
if __name__ == "__main__":
    fake_dir = 'exports/enhanced'
    real_dir = '../../dataset/TESTCTALIGNED'
    output_excel = './testsimilarity_metrics.xlsx'
    
    results = evaluate_all(fake_dir,real_dir)
    df = save_metrics_to_excel(results,output_excel)

Evaluating:   0%|                                        | 0/70 [00:00<?, ?it/s]

Comparing fake: 0_1.nii  <-->  real: test-volume-0.nii


Evaluating:   1%|▍                               | 1/70 [00:05<06:07,  5.33s/it]

Comparing fake: 10_1.nii  <-->  real: test-volume-10.nii


Evaluating:   3%|▉                               | 2/70 [00:09<04:57,  4.37s/it]

...

Comparing fake: 8_1.nii  <-->  real: test-volume-8.nii


Evaluating:  99%|██████████████████████████████▌| 69/70 [03:51<00:03,  3.17s/it]

Comparing fake: 9_1.nii  <-->  real: test-volume-9.nii


Evaluating: 100%|███████████████████████████████| 70/70 [03:54<00:00,  3.35s/it]


✅ Saved metrics to Excel: ./testsimilarity_metrics.xlsx
